In [1]:
import pyspark
import h2o
import pysparkling

print("Spark:", pyspark.__version__)
print("PySparkling:", pysparkling.__version__)
print("H2O:", h2o.__version__)

Spark: 3.5.1
PySparkling: 3.46.0.6-1-3.5
H2O: 3.46.0.6


In [1]:
from pyspark.sql import SparkSession
from ai.h2o.sparkling import H2OContext, H2OConf

import os, sys
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

spark = (
    SparkSession.builder
    .appName("H2O_Spark_35")
    .config("spark.driver.memory", "8g")
    .config("spark.executor.memory", "4g")
    .config("spark.ext.h2o.client.language", "python")
    # Jeśli pojawi się ClassNotFoundException, odkomentuj:
    # .config("spark.jars.packages", "ai.h2o:sparkling-water-package_2.12:3.46.0.6-1-3.5")
    .getOrCreate()
)

# <<< KLUCZ >>>: tworzysz H2OConf (bez argumentów!) i przekazujesz go do getOrCreate
conf = H2OConf().set("spark.ext.h2o.client.language", "python")

hc = H2OContext.getOrCreate(conf)   # <-- NIE przekazujesz tu `spark`

import h2o
print("H2O:", h2o.cluster().version)


Connecting to H2O server at http://host.docker.internal:54323 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,11 secs
H2O_cluster_timezone:,Europe/Belgrade
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 year and 4 days
H2O_cluster_name:,sparkling-water-Paweł_local-1762457130651
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.46.0.6-1-3.5
 * H2O name: sparkling-water-Paweł_local-1762457130651
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,192.168.50.103,54321)
  ------------------------

  Open H2O Flow in browser: http://host.docker.internal:54323 (CMD + click in Mac OSX)

    
H2O: 3.46.0.6


In [3]:
# === Ładowanie danych ze scikit-learn ===
import pandas as pd
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
pdf = pd.DataFrame(data.data, columns=[c.replace(" ", "_") for c in data.feature_names])

# target: 0=malignant, 1=benign -> nazwijmy klasy jawnie jako stringi
pdf["target"] = pd.Series(data.target).map({0: "malignant", 1: "benign"})

# === pandas -> Spark ===
sdf = spark.createDataFrame(pdf)

print("Rekordy:", sdf.count())


Rekordy: 569


In [4]:
sdf.printSchema()

root
 |-- mean_radius: double (nullable = true)
 |-- mean_texture: double (nullable = true)
 |-- mean_perimeter: double (nullable = true)
 |-- mean_area: double (nullable = true)
 |-- mean_smoothness: double (nullable = true)
 |-- mean_compactness: double (nullable = true)
 |-- mean_concavity: double (nullable = true)
 |-- mean_concave_points: double (nullable = true)
 |-- mean_symmetry: double (nullable = true)
 |-- mean_fractal_dimension: double (nullable = true)
 |-- radius_error: double (nullable = true)
 |-- texture_error: double (nullable = true)
 |-- perimeter_error: double (nullable = true)
 |-- area_error: double (nullable = true)
 |-- smoothness_error: double (nullable = true)
 |-- compactness_error: double (nullable = true)
 |-- concavity_error: double (nullable = true)
 |-- concave_points_error: double (nullable = true)
 |-- symmetry_error: double (nullable = true)
 |-- fractal_dimension_error: double (nullable = true)
 |-- worst_radius: double (nullable = true)
 |-- worst_

In [5]:
sdf.groupBy("target").count().show()

+---------+-----+
|   target|count|
+---------+-----+
|malignant|  212|
|   benign|  357|
+---------+-----+



In [6]:
# === Spark -> H2O, split, AutoML ===
hf = hc.asH2OFrame(sdf, "breast_cancer_hf")
response = "target"
hf[response] = hf[response].asfactor()

In [7]:
# train/valid/test: 70/15/15
train, valid, test = hf.split_frame(ratios=[0.70, 0.15], seed=42)

In [8]:
from h2o.automl import H2OAutoML
x = [c for c in hf.columns if c != response]

In [9]:
aml = H2OAutoML(
    max_runtime_secs=600,        # podbij jeśli chcesz więcej
    seed=42,
    sort_metric="AUC",
    nfolds=5,
    balance_classes=True,
    keep_cross_validation_predictions=True,
    keep_cross_validation_models=False
)
aml.train(x=x, y=response, training_frame=train, leaderboard_frame=valid)

AutoML progress: |
20:27:44.903: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_3_AutoML_1_20251106_202744_model_7


Status of Neuron Layers: predicting target, 2-class classification, bernoulli distribution, CrossEntropy loss, 23 502 weights/biases, 287,1 KB, 3 869 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  ----------------------  -------------------  ----------------------  ---------------------
    1        30       Input             5.0
    2        100      RectifierDropout  20.0       0.0   0.0   0.002800525306297156   0.00218180101364851    0.0         0.004193515759397997    0.12249171733856201  0.49694637657230983     0.01813054084777832
    3        100      RectifierDropout  20.0       0.0   0.0   0.0029124529346241618  0.0016495459713041782  0.0         -0.0025060537991128513  0.10133090615272522  0.988080772774948       0.021746598184108734
    4        100      RectifierDropout  20.0       0.0   0.0   0.008165533409197815   0.04072083532810211    0.0         -0.0018228730640281355  0.10161042213439941  0.9952035116998446      0.0071641262620687485
    5        2        Softmax                      0.0   0.0   0.0014129996625706553  0.0016394471749663353  0.0         -0.021615948712569662   0.5435266494750977   -2.969249798976878e-07  0.0037399884313344955

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.015985767826150646
RMSE: 0.12643483628395555
LogLoss: 0.05478899666268506
Mean Per-Class Error: 0.01242283950617284
AUC: 0.9989883401920439
AUCPR: 0.9990418833061593
Gini: 0.9979766803840877

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.8571584943466658
           benign    malignant    Error    Rate
---------  --------  -----------  -------  -----------
benign     237       3            0.0125   (3.0/240.0)
malignant  3         240          0.0123   (3.0/243.0)
Total      240       243          0.0124   (6.0/483.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.857158     0.987654  37
max f2                       0.857158     0.987654  37
max f0point5                 0.994863     0.991489  30
max accuracy                 0.857158     0.987578  37
max precision                1            1         0
max recall                   0.0241461    1         56
max specificity              1            1         0
max absolute_mcc             0.857158     0.975154  37
max min_per_class_accuracy   0.857158     0.9875    37
max mean_per_class_accuracy  0.857158     0.987577  37
max tns                      1            240       0
max fns                      1            72        0
max fps                      1.48885e-11  240       276
max tps                      0.0241461    243       56
max tnr                      1            1         0
max fnr                      1            0.296296  0
max fpr                      1.48885e-11  1         276
max tpr                      0.0241461    1         56

Gains/Lift Table: Avg response rate: 50,31 %, avg score: 51,20 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0517598              

In [18]:
lb = aml.leaderboard
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_grid_3_AutoML_1_20251106_202744_model_7,0.98539,0.284253,0.984492,0.030303,0.213946,0.0457731
DeepLearning_grid_1_AutoML_1_20251106_202744_model_4,0.983225,0.65517,0.980432,0.0481602,0.218918,0.0479253
DeepLearning_grid_2_AutoML_1_20251106_202744_model_4,0.982684,0.487211,0.978978,0.0606061,0.234661,0.0550659
DeepLearning_grid_2_AutoML_1_20251106_202744_model_19,0.982143,0.427724,0.978516,0.0606061,0.233003,0.0542902
DeepLearning_grid_1_AutoML_1_20251106_202744_model_46,0.982143,0.390101,0.978978,0.0543831,0.229549,0.0526926
DeepLearning_grid_1_AutoML_1_20251106_202744_model_3,0.981602,0.518414,0.980142,0.0392316,0.207537,0.0430715
DeepLearning_grid_2_AutoML_1_20251106_202744_model_3,0.981602,0.387062,0.980696,0.0454545,0.223225,0.0498296
DeepLearning_grid_3_AutoML_1_20251106_202744_model_3,0.981061,0.335981,0.980361,0.0454545,0.228856,0.0523752
DeepLearning_grid_3_AutoML_1_20251106_202744_model_4,0.980519,0.437478,0.975459,0.0570887,0.229369,0.0526102
DeepLearning_grid_1_AutoML_1_20251106_202744_model_9,0.980519,0.457216,0.977168,0.0419372,0.231944,0.053798


In [19]:
lb_pd = lb.as_data_frame()
lb_pd = lb_pd[~lb_pd['model_id'].str.contains("Deep")]
lb_pd

c:\REPO\studia\venv\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
10,StackedEnsemble_AllModels_3_AutoML_1_20251106_...,0.980519,0.173599,0.976511,0.063312,0.214761,0.046122
13,StackedEnsemble_AllModels_4_AutoML_1_20251106_...,0.979978,0.169734,0.977459,0.060606,0.209964,0.044085
14,StackedEnsemble_BestOfFamily_4_AutoML_1_202511...,0.979978,0.298617,0.977233,0.048160,0.211552,0.044754
15,StackedEnsemble_BestOfFamily_2_AutoML_1_202511...,0.979978,0.203917,0.975631,0.045455,0.204905,0.041986
25,GLM_1_AutoML_1_20251106_202744,0.978355,0.177240,0.978100,0.045455,0.206122,0.042486
29,StackedEnsemble_AllModels_1_AutoML_1_20251106_...,0.977814,0.204458,0.974168,0.045455,0.203135,0.041264
30,StackedEnsemble_BestOfFamily_5_AutoML_1_202511...,0.977814,0.205809,0.975258,0.045455,0.202698,0.041087
33,StackedEnsemble_BestOfFamily_7_AutoML_1_202511...,0.977273,0.224716,0.974965,0.045455,0.205964,0.042421
38,StackedEnsemble_Best1000_1_AutoML_1_20251106_2...,0.976732,0.181055,0.975149,0.057089,0.214667,0.046082
41,StackedEnsemble_BestOfFamily_6_AutoML_1_202511...,0.976461,0.194215,0.977271,0.045455,0.221033,0.048855


In [13]:
leader = aml.leader
print("Leader:", leader.algo, leader.model_id)

Leader: deeplearning DeepLearning_grid_3_AutoML_1_20251106_202744_model_7


In [20]:
# === Ocena na teście + próg pod F1 ===
perf = leader.model_performance(test_data=test)
print("AUC(test)   :", perf.auc())
print("AUCPR(test) :", perf.aucpr())
print("LogLoss(test):", perf.logloss())

AUC(test)   : 1.0
AUCPR(test) : 1.0
LogLoss(test): 0.05159368900882523


In [21]:
best_thr_f1 = perf.find_threshold_by_max_metric("f1")
print("Best threshold (F1):", best_thr_f1)
print(perf.confusion_matrix(metrics="f1"))

Best threshold (F1): 0.9999276132390292
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999276132390292
           benign    malignant    Error    Rate
---------  --------  -----------  -------  ----------
benign     61        0            0        (0.0/61.0)
malignant  0         28           0        (0.0/28.0)
Total      61        28           0        (0.0/89.0)


In [22]:
# === Predykcje + powrót do Spark ===
preds_h2o = leader.predict(test)         # kolumny: p0/p1/predict (dla binarnej)
preds_spark = hc.asSparkFrame(preds_h2o)
preds_spark.show(5)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
+---------+--------------------+------------------+
|  predict|              benign|         malignant|
+---------+--------------------+------------------+
|malignant|7.238676097078231E-5|0.9999276132390292|
|malignant|9.981415322312201E-8|0.9999999001858467|
|malignant|1.574546262827487...|0.9999998425453737|
|malignant|4.211755889030553...|0.9999999999999958|
|malignant|2.828283736897204E-7|0.9999997171716263|
+---------+--------------------+------------------+
only showing top 5 rows



In [23]:
# === Zapis modelu (H2O bin + MOJO) ===
import os, h2o
outdir = r"C:\REPO\studia\PRZETWARZAZBIOROW20252026\P2 Pysparkling\models\h2o_breast_cancer"
os.makedirs(outdir, exist_ok=True)

bin_path  = h2o.save_model(model=leader, path=outdir, force=True)
mojo_path = leader.download_mojo(path=outdir, get_genmodel_jar=True)
print("Saved BIN :", bin_path)
print("Saved MOJO:", mojo_path)

Saved BIN : C:\REPO\studia\PRZETWARZAZBIOROW20252026\P2 Pysparkling\models\h2o_breast_cancer\DeepLearning_grid_3_AutoML_1_20251106_202744_model_7
Saved MOJO: C:\REPO\studia\PRZETWARZAZBIOROW20252026\P2 Pysparkling\models\h2o_breast_cancer\DeepLearning_grid_3_AutoML_1_20251106_202744_model_7.zip


In [24]:
aml = H2OAutoML(
    max_runtime_secs=600,        
    seed=42,
    sort_metric="AUC",
    nfolds=5,
    balance_classes=True,
    keep_cross_validation_predictions=True,
    keep_cross_validation_models=False,
    exclude_algos=["DeepLearning", 'XGBoost']
)
aml.train(x=x, y=response, training_frame=train, leaderboard_frame=valid)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/3
# GBM base models (used / total),1/1
# GLM base models (used / total),1/1
# DRF base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [25]:
lb = aml.leaderboard
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_2_AutoML_2_20251106_204540,0.979978,0.203917,0.975631,0.0454545,0.204905,0.0419862
GBM_grid_1_AutoML_2_20251106_204540_model_228,0.979437,0.219746,0.974835,0.0454545,0.198551,0.0394227
GBM_grid_1_AutoML_2_20251106_204540_model_120,0.978896,0.282075,0.97252,0.0543831,0.216169,0.0467292
GBM_grid_1_AutoML_2_20251106_204540_model_161,0.978355,0.291956,0.972674,0.0543831,0.204215,0.0417038
GLM_1_AutoML_2_20251106_204540,0.978355,0.17724,0.9781,0.0454545,0.206122,0.0424861
GBM_grid_1_AutoML_2_20251106_204540_model_68,0.978355,0.239388,0.972756,0.0543831,0.213768,0.0456969
StackedEnsemble_AllModels_1_AutoML_2_20251106_204540,0.977814,0.204458,0.974168,0.0454545,0.203135,0.0412638
GBM_grid_1_AutoML_2_20251106_204540_model_237,0.977273,0.232757,0.971302,0.0543831,0.220613,0.0486701
StackedEnsemble_AllModels_3_AutoML_2_20251106_204540,0.976732,0.224431,0.973168,0.0454545,0.250459,0.0627298
GBM_grid_1_AutoML_2_20251106_204540_model_210,0.976732,0.24415,0.970276,0.0606061,0.214569,0.0460398


In [26]:
leader = aml.leader
print("Leader:", leader.algo, leader.model_id)

Leader: stackedensemble StackedEnsemble_BestOfFamily_2_AutoML_2_20251106_204540


In [27]:
# === Ocena na teście + próg pod F1 ===
perf = leader.model_performance(test_data=test)
print("AUC(test)   :", perf.auc())
print("AUCPR(test) :", perf.aucpr())
print("LogLoss(test):", perf.logloss())

AUC(test)   : 1.0
AUCPR(test) : 1.0
LogLoss(test): 0.030195947664521938


In [28]:
best_thr_f1 = perf.find_threshold_by_max_metric("f1")
print("Best threshold (F1):", best_thr_f1)
print(perf.confusion_matrix(metrics="f1"))

Best threshold (F1): 0.9072210702480294
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9072210702480294
           benign    malignant    Error    Rate
---------  --------  -----------  -------  ----------
benign     61        0            0        (0.0/61.0)
malignant  0         28           0        (0.0/28.0)
Total      61        28           0        (0.0/89.0)


In [29]:
# === Predykcje + powrót do Spark ===
preds_h2o = leader.predict(test)         # kolumny: p0/p1/predict (dla binarnej)
preds_spark = hc.asSparkFrame(preds_h2o)
preds_spark.show(5)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
+---------+--------------------+------------------+
|  predict|              benign|         malignant|
+---------+--------------------+------------------+
|malignant| 0.09277892975197055|0.9072210702480294|
|malignant|0.006204363178459...|0.9937956368215404|
|malignant|1.466680307093959...|0.9998533319692906|
|malignant| 3.12627168597146E-9|0.9999999968737283|
|malignant|0.005996768160698607|0.9940032318393014|
+---------+--------------------+------------------+
only showing top 5 rows



In [31]:
# === Zapis modelu (H2O bin + MOJO) ===
import os, h2o
outdir = r"C:\REPO\studia\PRZETWARZAZBIOROW20252026\P2 Pysparkling\models\h2o_breast_cancer\nodeeplarning_xgboost"
os.makedirs(outdir, exist_ok=True)

bin_path  = h2o.save_model(model=leader, path=outdir, force=True)
mojo_path = leader.download_mojo(path=outdir, get_genmodel_jar=True)
print("Saved BIN :", bin_path)
print("Saved MOJO:", mojo_path)


Saved BIN : C:\REPO\studia\PRZETWARZAZBIOROW20252026\P2 Pysparkling\models\h2o_breast_cancer\nodeeplarning_xgboost\StackedEnsemble_BestOfFamily_2_AutoML_2_20251106_204540
Saved MOJO: C:\REPO\studia\PRZETWARZAZBIOROW20252026\P2 Pysparkling\models\h2o_breast_cancer\nodeeplarning_xgboost\StackedEnsemble_BestOfFamily_2_AutoML_2_20251106_204540.zip
